In [5]:
import cv2
def on_trackbar(x):
    pass

img_src = cv2.imread('images/shape1.png',cv2.IMREAD_COLOR)
height, width = img_src.shape[:2]

img_gray = cv2.cvtColor(img_src, cv2.COLOR_BGR2GRAY)

##################### 트랙바 구현 #######################
# 1. 트랙바가 포함되어 있는 윈도우를 생성
cv2.namedWindow("TrackBar")  # 윈도우의 이름을 계속 사용

# 2. 트랙바를 몇개만들것인가 정함
# cv2.createTrackbar("트랙바이름", "윈도우이름", 최소값,최대값, 호출함수)
# 트랙바를 움직이면 바로 적용될때
cv2.createTrackbar("threshold", "TrackBar", 0,255, on_trackbar) 
# 특별한 이벤트를 적용하지 않을때
cv2.createTrackbar("maxValue", "TrackBar", 0,255, lambda x : x) 

# 3. 트랙바의 초기값 설정
# cv2.setTrackbarPos("트랙바이름","윈도우이름", 초기값) 
cv2.setTrackbarPos('threshold',"TrackBar", 127)
cv2.setTrackbarPos('maxValue',"TrackBar", 255)

# 4. 트랙바는 무한루프를 통해 연속적으로 변해야 함
while cv2.waitKey(1) != ord('q'):
    # 5. 트랙바 값을 변수로 대입
    # 변수이름 = cv2.getTrackbarPos("트랙바이름","윈도우이름")
    thr_value = cv2.getTrackbarPos('threshold','TrackBar')
    mav_value = cv2.getTrackbarPos('maxValue','TrackBar')
    ret, img_dst = cv2.threshold(img_gray, thr_value, mav_value, cv2.THRESH_BINARY)
    img_dst = cv2.bitwise_not(img_dst)
    cv2.imshow('TrackBar',img_dst)
########################################################


cv2.destroyAllWindows()

## 흐림(Blur. 스무딩이라고도 부른다) : 사진 속 먼지 등 불필요한 노이즈(보다는 아웃라이어?) 제거

##원래 이미지 사이즈에 맞춰 블러필터의 크기도 맞춰야(이미지가 크다면 필터도 크게)
- 노이즈(먼지등의 조그마한 난반사된 물체)를 제거하기 위해서 영상을 흐리게 만듬
- 원본영상의 크기에 따라 주변픽셀의 영역을 적당히 지정해야함
- img_dst = cv2.blur(img_src, anchor=(-1,-1), borderType=cv2.BORDER_DEFAULT) 
=> 평균을 내서 어떤 픽셀에 붙일까? 보통은 필터의 가운데에 해당되는 원본 이미지 영역에 붙인다. 그런데 원본이미지와 이미지 밖을 걸쳐야 할 때, 경계부분 처리하는 게 BORDER인자. 디폴트는 그냥 끝값 이용

In [14]:
import cv2
img_src = cv2.imread('images/hoshino.jpg',cv2.IMREAD_COLOR)
height, width = img_src.shape[:2]
###########################################
# 블러 처리
#img_dst = cv2.blur(img_src, (9,9), anchor=(-1,-1),borderType=cv2.BORDER_DEFAULT) #(7,7)은 필터사이즈. 보통 홀수. 앵커는 필터계산후 넣을 위치. -1,-1이면 중심점.
img_dst = cv2.GaussianBlur(img_src, (9,9), 0)


############################################
cv2.imshow('src',img_src)
cv2.imshow('blur',img_dst)
cv2.waitKey()
cv2.destroyAllWindows()

## 경계선 추출
- edge추출
- Sobel
- Laplacian
- Canny (얘가 젤 좋다)

In [ ]:
# sobel 에지

import cv2

scale = 1
delta = 0
ddepth = cv2.CV_16S

img_src = cv2.imread('images/hisame.jpg', cv2.IMREAD_COLOR)
img_src = cv2.GaussianBlur(img_src, (3, 3), 0)
img_gray = cv2.cvtColor(img_src, cv2.COLOR_BGR2GRAY)

# 소벨 가로마스크 적용    
grad_x = cv2.Sobel(img_gray, ddepth, 1, 0, ksize=3, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT) #1,0 dx방향
grad_y = cv2.Sobel(img_gray, ddepth, 0, 1, ksize=3, scale=scale, delta=delta, borderType=cv2.BORDER_DEFAULT)

#음수가 나올 수 있으니
abs_grad_x = cv2.convertScaleAbs(grad_x)
abs_grad_y = cv2.convertScaleAbs(grad_y)


grad = cv2.addWeighted(abs_grad_x, 0.5, abs_grad_y, 0.5, 0)  #위에 두 그라디언트를 더하니까 색의 범위를 초과하지 않게 가중치0.5를 곱하여 머지

cv2.imshow('sobel', grad)
cv2.waitKey()
cv2.destroyAllWindows()
    

라플라시안 에지

In [20]:
import cv2 

ddepth = cv2.CV_16S
kernel_size = 3

img_src = cv2.imread('images/hisame.jpg', cv2.IMREAD_COLOR) # Load an image
img_src = cv2.GaussianBlur(img_src, (3, 3), 0)
img_gray = cv2.cvtColor(img_src, cv2.COLOR_BGR2GRAY)

img_dst = cv2.Laplacian(img_gray, ddepth, ksize=kernel_size)
abs_dst = cv2.convertScaleAbs(img_dst)
cv2.imshow('Laplace', abs_dst)
cv2.waitKey(0)
cv2.destroyAllWindows()

캐니 에지

In [28]:
import cv2

max_lowThreshold = 100
ratio = 3
kernel_size = 3
 
def CannyThreshold(val):
    low_threshold = val
    img_blur = cv2.blur(img_gray, (3,3))
    detected_edges = cv2.Canny(img_blur, low_threshold, max_lowThreshold, kernel_size)
    mask = detected_edges != 0
    img_dst = img_src * (mask[:,:,None].astype(img_src.dtype))
    cv2.imshow('CannyEdge', img_dst)
    cv2.imshow('src', img_src)
 
 
img_src = cv2.imread('images/hoshino.jpg',cv2.IMREAD_COLOR)
img_gray = cv2.cvtColor(img_src, cv2.COLOR_BGR2GRAY)
 
CannyThreshold(50)
cv2.waitKey()
cv2.destroyAllWindows()

In [24]:
import cv2
import numpy as np

max_lowThreshold = 100
ratio = 3
kernel_size = 3

def CannyThreshold(val):
    low_threshold = val
    img_blur = cv2.blur(img_gray, (3,3))
    detected_edges = cv2.Canny(img_blur, low_threshold, low_threshold*ratio, kernel_size)
    # More efficient masking using bitwise_and
    img_dst = cv2.bitwise_and(img_src, img_src, mask=detected_edges)
    cv2.imshow('CannyEdge', img_dst)

try:
    img_src = cv2.imread('images/hoshino.jpg', cv2.IMREAD_COLOR)
    if img_src is None:
        raise FileNotFoundError("Image not found.")

    img_gray = cv2.cvtColor(img_src, cv2.COLOR_BGR2GRAY)

    cv2.namedWindow('CannyEdge')
    cv2.createTrackbar('Min Threshold', 'CannyEdge', 0, max_lowThreshold, CannyThreshold)  # Add Trackbar

    CannyThreshold(0)  # Initial display

    cv2.waitKey(0)
    cv2.destroyAllWindows()

except FileNotFoundError as e:
    print(f"Error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")